In [20]:
import cv2 as cv
import numpy as np
import time

cap = cv.VideoCapture(0)

if (cap.isOpened() == False):
    print("Error opening video stream or file")

frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

_, frame1 = cap.read()
prev = cv.cvtColor(frame1, cv.COLOR_BGR2GRAY)
hsv_mask = np.zeros_like(frame1)
hsv_mask[..., 1] = 255


while(cap.isOpened()):
    _, frame2 = cap.read()
    next = cv.cvtColor(frame2, cv.COLOR_BGR2GRAY)

    # pyr_scale, levels, winsize, interations, poly_n, poly_sigma, flow
    flow = cv.calcOpticalFlowFarneback(prev, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    # Convert optical flow into Polar coordinates to get magnitude
    mag, ang = cv.cartToPolar(flow[..., 0], flow[..., 1])
    # Use a threshold, to only count the significant ones
    mag_thresholded = (mag > 20)
    
    display = frame2
    for row in range(4, display.shape[0], 8):
     for col in range(4, display.shape[1], 8):
         flowat = flow[row, col]
         cv.line(display,
               (col, row),
               (col + round(flowat[1]), row + round(flowat[0])),
               (0, 255, 0))

    cv.imshow('Frame', display)
    key = cv.waitKey(40)
    
    percent = mag_thresholded.sum() / (frame_width * frame_height)
    if percent > 0.02:
        current_time = time.time()
        # Save the current color frame in a file (with a counter or time stamp)
        cv.imwrite(f'{current_time}.png', display)
        
    if key & 0xFF == ord('q'):
        break

cap.release()
cv.destroyAllWindows()
